In [12]:
import pandas as pd 
import numpy as np  
import re
import random
import json 

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementClickInterceptedException, TimeoutException
from time import sleep

from bs4 import BeautifulSoup
import time
import itertools
from itertools import product

import os
os.environ['WDM_SSL_VERIFY'] = '0'

path = 'C:/Users/GMEU0170/OneDrive - Hyundai Motor Europe/GME_CD/Documents/2.__Code/_.py/web_scraping/selenium/web_scraping_git/DOM_Web_Scraping/'

In [2]:
# Set up Chrome options (headless mode is optional)
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_experimental_option(
    "prefs", {"profile.managed_default_content_settings.images": 2}
)

# Use webdriver-manager to install and manage ChromeDriver
service = Service(ChromeDriverManager().install())
# service = Service(path+'chromedriver/chromedriver-win64/chromedriver.exe')

c:\Users\GMEU0170\AppData\Local\anaconda3\envs\scrapy\lib\site-packages\urllib3\connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host 'googlechromelabs.github.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\GMEU0170\AppData\Local\anaconda3\envs\scrapy\lib\site-packages\urllib3\connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host 'googlechromelabs.github.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\GMEU0170\AppData\Local\anaconda3\envs\scrapy\lib\site-packages\urllib3\connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host 'googlechromelabs.github.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.i

In [3]:
# User-Agent list for rotation
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.1 Safari/605.1.15",
    # Add more user agents as needed
]

def get_driver(service):
    chrome_options = webdriver.ChromeOptions()
    # chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--blink-settings=imagesEnabled=false")
    ua = random.choice(user_agents)
    chrome_options.add_argument(f"user-agent={ua}")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option("useAutomationExtension", False)
    # Explicitly set the page load strategy to "normal"
    chrome_options.page_load_strategy = "normal"
    
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.maximize_window()
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    driver.implicitly_wait(10)
    return driver

# driver = get_driver(service)

In [7]:
driver = get_driver(service)

driver.get("https://www.bmw.de/de/konfigurator.html")
time.sleep(2)  # Wait for everything to load

js_find_and_click_cookie = """
    function findAndClickAccept(node) {
        if (!node) return false;

        // Check if it's a button or span containing the "Alle akzeptieren" text
        if ((node.tagName === 'BUTTON' || node.tagName === 'SPAN') && node.innerText.includes("Alle akzeptieren")) {
            node.click();
            return true;
        }

        // Traverse shadow DOM
        if (node.shadowRoot) {
            const found = findAndClickAccept(node.shadowRoot);
            if (found) return true;
        }

        // Traverse regular child elements
        for (const child of node.children || []) {
            const found = findAndClickAccept(child);
            if (found) return true;
        }

        return false;
    }
    return findAndClickAccept(document.body);
"""

# Execute the JavaScript in Selenium
clicked = driver.execute_script(js_find_and_click_cookie)

if clicked:
    print("✅ 'Alle akzeptieren' button clicked.")
else:
    print("❌ Could not find 'Alle akzeptieren' button.")

sleep(2)
    
# JavaScript to extract all <a class="cmp-allmodelscard__link"> inside shadow roots
js_script = """
    function deepQuerySelectorAll(selector) {
        const results = [];

        function traverse(node) {
            if (node.shadowRoot) {
                traverse(node.shadowRoot);
            }

            const elements = node.querySelectorAll(selector);
            if (elements.length) {
                elements.forEach(el => results.push(el));
            }

            // Recurse into child nodes
            node.children && [...node.children].forEach(traverse);
        }

        traverse(document);
        return results;
    }

    return deepQuerySelectorAll("a.cmp-allmodelscard__link").map(a => a.href);
"""

# Run the JS to get all links
car_links = driver.execute_script(js_script)

# Print the result
print(f"Found {len(car_links)} car config URLs:")
url_set = set()
for link in car_links:
    if link not in url_set:
        url_set.add(link)
        
# Close the driver
driver.quit()

✅ 'Alle akzeptieren' button clicked.
Found 5304 car config URLs:


In [8]:
url_list = list(url_set) 
url_list_bmw = [url for url in url_list if url not in url_list_manual]

len(url_list), len(url_list_bmw)

NameError: name 'url_list_manual' is not defined

In [9]:
# Func to get progress of tool
def load_progress(progress_file):
    """Load saved progress from a file"""
    if os.path.exists(progress_file):
        with open(progress_file, "r") as file:
            return json.load(file)
    else:
        return {"last_processed_index": 0}  # Default to starting from the beginning

def save_progress(progress_file, progress_data):
    """Save current progress to a file"""
    with open(progress_file, "w") as file:
        json.dump(progress_data, file)

In [10]:
### set up progress file
progress_file = path + 'output/bmw/' + 'progress.json'
progress = load_progress(progress_file)
start_index = progress["last_processed_index"]

start_index

0

In [ ]:
### set up progress file
progress_file = path + 'output/bmw/' + 'progress.json'
progress = load_progress(progress_file)
start_index = progress["last_processed_index"]

for url in url_list_bmw[:2]:
    
    try:
                
        driver = get_driver(service)

        # url = "https://configure.bmw.de/de_DE/configure/I20"
        # url = "https://configure.bmw.de/de_DE/configure/U10/51GM/FBLAT,P0C68,S01EE,S0230,S02PA,S02VB,S02VD,S0302,S0428,S0478,S0494,S04GN,S04T2,S04U9,S04V1,S0548,S05AV,S05DM,S0654,S06AE,S06AF,S06PA,S0760,S0801,S0851,S0879,S08R3,S08R9,S08TF/SE000001"
        driver.get(url)

        sleep(1)

        js_find_and_click_cookie = """
            function findAndClickAccept(node) {
                if (!node) return false;

                // Check if it's a button or span containing the "Alle akzeptieren" text
                if ((node.tagName === 'BUTTON' || node.tagName === 'SPAN') && node.innerText.includes("Alle akzeptieren")) {
                    node.click();
                    return true;
                }

                // Traverse shadow DOM
                if (node.shadowRoot) {
                    const found = findAndClickAccept(node.shadowRoot);
                    if (found) return true;
                }

                // Traverse regular child elements
                for (const child of node.children || []) {
                    const found = findAndClickAccept(child);
                    if (found) return true;
                }

                return false;
            }
            return findAndClickAccept(document.body);
        """

        # Execute the JavaScript in Selenium
        clicked = driver.execute_script(js_find_and_click_cookie)

        if clicked:
            print("✅ 'Alle akzeptieren' button clicked.")
        else:
            print("❌ Could not find 'Alle akzeptieren' button.")

        sleep(3)

        body = driver.find_element(By.TAG_NAME, "body")
        all_text = body.text

        js_find_and_click = """
            function findAndClickButton(node) {
                if (!node) return null;

                if (node.tagName === "BUTTON" && node.innerText.includes("Finanzieren & Leasen")) {
                    node.click();
                    return true;
                }

                if (node.shadowRoot) {
                    let found = findAndClickButton(node.shadowRoot);
                    if (found) return true;
                }

                for (const child of node.children || []) {
                    let found = findAndClickButton(child);
                    if (found) return true;
                }

                return false;
            }
            return findAndClickButton(document.body);
            """

        clicked = driver.execute_script(js_find_and_click)
        if clicked:
            print("✅ Button clicked successfully!")
        else:
            print("❌ Button not found!")
            
        sleep(2)
        
        ### WORKING
        results = driver.execute_async_script("""
            const done = arguments[0];

            function findElementByAriaLabel(root, label) {
                if (!root) return null;
                if (root.getAttribute && root.getAttribute("aria-label") === label) return root;

                if (root.shadowRoot) {
                    const result = findElementByAriaLabel(root.shadowRoot, label);
                    if (result) return result;
                }

                for (const child of root.children || []) {
                    const result = findElementByAriaLabel(child, label);
                    if (result) return result;
                }
                return null;
            }

            async function selectAllOptions() {
                const select = findElementByAriaLabel(document.body, "Weitere Finanzierungs- und Leasingbeispiele:");
                if (!select) {
                    done({error: "Select element not found"});
                    return;
                }

                const results = [];

                for (let i = 0; i < select.options.length; i++) {
                    select.selectedIndex = i;
                    select.dispatchEvent(new Event("input", { bubbles: true }));
                    select.dispatchEvent(new Event("change", { bubbles: true }));

                    await new Promise(resolve => setTimeout(resolve, 1000));

                    results.push({
                    index: i,
                    optionText: select.options[i].textContent.trim(),
                    optionValue: select.options[i].value
                    });
                }

                done(results);
            }

            selectAllOptions();
        """)

        if isinstance(results, dict) and "error" in results:
            print("Error:", results["error"])
        else:
            for res in results:
                print(f"Option {res['index']}: {res['optionText']} (value: {res['optionValue']})")


        # Step 1: Find main dropdown inside shadow roots by aria-label
        get_main_dropdown_js = """
        function findElementByAriaLabel(root, label) {
            if (!root) return null;
            if (root.getAttribute && root.getAttribute("aria-label") === label) return root;
            if (root.shadowRoot) {
                const result = findElementByAriaLabel(root.shadowRoot, label);
                if (result) return result;
            }
            for (const child of root.children || []) {
                const result = findElementByAriaLabel(child, label);
                if (result) return result;
            }
            return null;
        }
        return findElementByAriaLabel(document.body, "Weitere Finanzierungs- und Leasingbeispiele:");
        """

        main_dropdown = driver.execute_script(get_main_dropdown_js)

        # Step 2: Get all options from the main dropdown
        option_elements = driver.execute_script("return Array.from(arguments[0].options);", main_dropdown)

        print(f"Found {len(option_elements)} options in main dropdown.")

        # Step 3: For each option, click, wait for UI update, then fetch related dropdown options
        all_options_data = []

        for i in range(len(option_elements)):
            # Click the main dropdown to open it
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", main_dropdown)
            main_dropdown.click()
            time.sleep(0.5)  # short pause
            
            # Select the ith option by click (simulate user action)
            option_to_click_js = """
            const select = arguments[0];
            const index = arguments[1];
            select.selectedIndex = index;
            select.dispatchEvent(new Event('change', { bubbles: true }));
            return select.options[index].value;
            """
            value_selected = driver.execute_script(option_to_click_js, main_dropdown, i)
            print(f"Clicked option {i} with value {value_selected}")

            time.sleep(2)  # wait for UI update (adjust as needed)
            
            # Step 4: Find related selects (Laufzeit, Laufleistung p.a., and main dropdown itself)
            js_find_target_selects = """
                function findMatchingSelects(node, result = []) {
                    if (!node) return result;
                    if (node.tagName === "SELECT") {
                        const label = node.getAttribute("aria-label");
                        if (label === "Laufzeit" || label === "Laufleistung p.a." || label === "Weitere Finanzierungs- und Leasingbeispiele:") {
                            result.push(node);
                        }
                    }
                    if (node.shadowRoot) findMatchingSelects(node.shadowRoot, result);
                    for (const child of node.children || []) {
                        findMatchingSelects(child, result);
                    }
                    return result;
                }
                return findMatchingSelects(document.body);
            """
            selects_after_click = driver.execute_script(js_find_target_selects)

            # Step 5: Extract option values and texts from each select found
            data_for_this_option = {"selected_main_value": value_selected, "dropdowns": []}

            for sel in selects_after_click:
                label = driver.execute_script("return arguments[0].getAttribute('aria-label');", sel)
                options = driver.execute_script("""
                    return Array.from(arguments[0].options).map(o => ({text: o.textContent.trim(), value: o.value}));
                """, sel)
                
                if label == "Laufleistung p.a." and options:
                    options = [options[-1]]  # Keep only the last option
                
                data_for_this_option["dropdowns"].append({"label": label, "options": options})

            all_options_data.append(data_for_this_option)

        # Print the gathered data nicely
        for entry in all_options_data:
            print(f"\nFor main dropdown value: {entry['selected_main_value']}")
            for dropdown in entry["dropdowns"]:
                print(f"  Dropdown: {dropdown['label']}")
                for opt in dropdown["options"]:
                    print(f"    - {opt['text']} (value: {opt['value']})")


        js_click_ok_button = """
            function clickOkButton(node) {
                if (!node) return false;

                if (node.tagName === "BUTTON" && node.classList.contains("sf-confi-commit")) {
                    node.click();
                    return true;
                }

                if (node.shadowRoot) {
                    const result = clickOkButton(node.shadowRoot);
                    if (result) return true;
                }

                for (const child of node.children || []) {
                    const result = clickOkButton(child);
                    if (result) return true;
                }

                return false;
            }
            return clickOkButton(document.body);
        """

        data_rows = []

        # Add model name from current context
        # all_text = driver.find_element(By.TAG_NAME, "body").text
        lines = [line.strip() for line in all_text.strip().splitlines() if line.strip()]   
        model_name = next((line for line in lines if "BMW" in line), "Unknown")

        # Loop over each configuration group from earlier shadow DOM scan
        for config_group in all_options_data:
            main_value = config_group["selected_main_value"]

            # Step 1: Build all_options array from the dropdowns
            dropdowns = config_group["dropdowns"]

            # Respect the order: main > laufzeit > laufleistung
            dropdown_order = ["Weitere Finanzierungs- und Leasingbeispiele:", "Laufzeit", "Laufleistung p.a."]
            all_options = []
            for label in dropdown_order:
                dropdown = next((d for d in dropdowns if d["label"] == label), None)
                if dropdown:
                    all_options.append([opt["value"] for opt in dropdown["options"]])

            # Step 2: Loop through all combinations of options
            for combo in product(*all_options):
                
                try:
                    clicked = driver.execute_script(js_click_ok_button)
                    if clicked:
                        print("✅ Dismissed OK popup.")
                        sleep(1)
                except Exception as e:
                    print(f"⚠️ Failed to click OK: {e}")
                
                print(f"\n🔁 Trying combo: {combo}")

                # Re-fetch the target selects to avoid stale references
                select_elements = driver.execute_script(js_find_target_selects)

                for i, value in enumerate(combo):
                    try:
                        # ❗ Re-fetch just before using it to avoid stale elements
                        select_elements = driver.execute_script(js_find_target_selects)
                        sel = select_elements[i]

                        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", sel)
                        sel.click()
                        sleep(1)

                        # Find index of desired value
                        index = driver.execute_script("""
                            const options = Array.from(arguments[0].options);
                            return options.findIndex(opt => opt.value === arguments[1]);
                        """, sel, value)

                        if index == -1:
                            print(f"❌ Value '{value}' not found in select[{i}]")
                            continue

                        # Use ARROW keys to move to correct option
                        actions = ActionChains(driver)
                        actions.send_keys(Keys.HOME)
                        for _ in range(index):
                            actions.send_keys(Keys.DOWN)
                        actions.send_keys(Keys.RETURN).perform()
                        sleep(1.5)
                    except Exception as e:
                        print(f"⚠️ Error selecting option {value} in combo {combo}: {e}")
                        continue

                # Step 3: Extract result values
                js_extract_label_and_value = """
                    function findNumbersWithLabels(node) {
                        let results = [];
                        if (node.tagName === "SF-CF-NUMBER") {
                            let span = node.querySelector("span.sf-cf-number");
                            if (span) {
                                results.push({
                                    parameterId: node.getAttribute("data-parameterid"),
                                    value: span.textContent.trim()
                                });
                            }
                        }
                        if (node.shadowRoot) {
                            results = results.concat(findNumbersWithLabels(node.shadowRoot));
                        }
                        for (const child of node.children || []) {
                            results = results.concat(findNumbersWithLabels(child));
                        }
                        return results;
                    }
                    return findNumbersWithLabels(document.body);
                """
                results = driver.execute_script(js_extract_label_and_value)

                print(f"📦 Model: {model_name} - Option {combo}")
                row_data = {"Model": f"{model_name} - Option {combo}"}

                desired_ids = {
                    "Price_Leasing", "Term", "DepositAmountLeasing",
                    "MonthlyPaymentWithoutServices", "Mileage", "TotalPaymentWithoutServices",
                    "Price", "CreditAmount", "DepositAmount", "NominalInterestRate", "InterestRate",
                    "MonthlyPayment_Ziel", "TotalCredit", "BallonAmount"
                }

                for item in results:
                    param_id = item['parameterId']
                    value = item['value']
                    if item["parameterId"] != None:
                        row_data[param_id] = value
                        print(f"  • {item['parameterId']}: {item['value']}")
                    # if item["parameterId"] in desired_ids:
                    #     row_data[item["parameterId"]] = item["value"]

                data_rows.append(row_data)

                # Save after each model for safety
                df = pd.DataFrame(data_rows)
                df['Country'] = 'Germany'
                df['Model_Name'] =  model_name
                df['URL'] = url
                columns = ["Country", "Model_Name", "URL"] + [col for col in df.columns if col not in ("Country", "Model_Name", "URL")]
                df = df[columns]
                df.to_csv(path + f'output/bmw/batch_2/{model_name}.csv', index=False)

                progress["last_processed_index"] = url_list_bmw.index(url) + 1
                save_progress(progress_file, progress)
            
        driver.quit()
    
    except Exception as e:
        print(f"[ERROR] Failed at index {i} ({url})\nReason: {e}")
        # traceback.print_exc()

        # Always close driver to prevent memory leaks
        try:
            driver.quit()
        except:
            pass

        # Optionally continue to next URL
        continue


✅ 'Alle akzeptieren' button clicked.
✅ Button clicked successfully!
Option 0: Das Ziel-Finanzierungsbeispiel der BMW Bank GmbH (value: 1004964)
Option 1: Das Leasingbeispiel der BMW Bank GmbH (value: 1008269)
Found 2 options in main dropdown.
Clicked option 0 with value 1004964
Clicked option 1 with value 1008269

For main dropdown value: 1004964
  Dropdown: Weitere Finanzierungs- und Leasingbeispiele:
    - Das Ziel-Finanzierungsbeispiel der BMW Bank GmbH (value: 1004964)
    - Das Leasingbeispiel der BMW Bank GmbH (value: 1008269)
  Dropdown: Laufzeit
    - 24 Monate (value: 24)
    - 30 Monate (value: 30)
    - 36 Monate (value: 36)
    - 42 Monate (value: 42)
    - 48 Monate (value: 48)
    - 54 Monate (value: 54)
    - 60 Monate (value: 60)

For main dropdown value: 1008269
  Dropdown: Weitere Finanzierungs- und Leasingbeispiele:
    - Das Ziel-Finanzierungsbeispiel der BMW Bank GmbH (value: 1004964)
    - Das Leasingbeispiel der BMW Bank GmbH (value: 1008269)
  Dropdown: Laufzeit